In [1]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_ollama.chat_models import ChatOllama
from pydantic import BaseModel, Field
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

In [2]:
embed_model = OllamaEmbeddings(base_url="http://localhost:11434", model="bge-m3:latest")
print(embed_model)
vectorstore = Chroma(collection_name="collection_01", persist_directory="./db/chroma_db_02", embedding_function=embed_model)
# retriever = vectorstore.as_retriever()
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={'k': 3, "fetch_k":5})
retriever

model='bge-m3:latest' base_url='http://localhost:11434' client_kwargs={}


VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001ADF1943F50>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 5})

In [3]:
# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

In [4]:
llm = ChatGroq(temperature=0, model_name= "llama-3.2-11b-text-preview")
# llm = ChatOllama(base_url="http://localhost:11434", model="llama3.2:latest")
structured_llm_grader = llm.with_structured_output(GradeDocuments)
structured_llm_grader

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001ADF313A6F0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001ADF32A0C80>, model_name='llama-3.2-11b-text-preview', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'GradeDocuments', 'description': 'Binary score for relevance check on retrieved documents.', 'parameters': {'properties': {'binary_score': {'description': "Documents are relevant to the question, 'yes' or 'no'", 'type': 'string'}}, 'required': ['binary_score'], 'type': 'object'}}}], 'tool_choice': {'type': 'function', 'function': {'name': 'GradeDocuments'}}}, config={}, config_factories=[])
| PydanticToolsParser(first_tool_only=True, tools=[<class '__main__.GradeDocuments'>])

In [5]:
# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)
retrieval_grader = grade_prompt | structured_llm_grader
retrieval_grader


ChatPromptTemplate(input_variables=['document', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You are a grader assessing relevance of a retrieved document to a user question. \n \n    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n\n    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n\n    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['document', 'question'], input_types={}, partial_variables={}, template='Retrieved document: \n\n {document} \n\n User question: {question}'), additional_kwargs={})])
| RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001ADF313

In [6]:
%timeit
question = """
Classification serves as verification system for a number of parties who have special interest in the safety
and quality of ships, such as:
— National authorities, who accept ships for registry, or let ships into their territorial waters, need assurance
that they are safe to sail on and represent a minimum hazard to their surroundings.
— Insurance underwriters require ships to be classed in order to give insurance.
— Owners, who need the technical standard of the rules as basis for building contracts and to document the
ships' standard when seeking insurance or financing, or when hiring out or selling the ship.
— Building yards and sub-contractors use the rules as a tool for design and construction, as required by their
client.
— Finance institutions use classification as a documented indicator of the
"""
docs = retriever.invoke(question)
docs

[Document(metadata={'File Name': 'DNV Rules for Classification of Ships _2016_08_Introduction to Ship Classification', 'File Path': '/content/drive/MyDrive/Rules/DNV/DNV Rules for Classification of Ships _2016_08_Introduction to Ship Classification.pdf', 'First Division': 'Rules', 'Page': 4, 'Second Division': 'DNV'}, page_content="This page explains DNV Rules for Classification of Ships _2016_08_Introduction to Ship Classification, that belongs to catogories of Rules and DNV./ndisclosed to any party, apart fro m the national authorities in volved, without the owners consent. The Society also undertakes all reporting to national authorities required in connection with the safety certificates. B 200 Who needs classification? 201  Classification serves as verification system for a number of parties who have special interest in the safety and quality of ships, such as: — National authorities, who accept ships for registry, or let ships into thei r territorial waters, need assurance that t

In [7]:
doc_txt = docs[2].page_content
doc_txt

'This page explains 00-part-1A-ships-jul24_Conditions of Classification, that belongs to catogories of Rules and ABS./nC H A P T E R 1 1 Classification Requirements for Naval Ships S E C T I O N 6 Naval Administration Requirements 1 Ship Specifications Ship specifications capture Naval Administration requirements that are contractually passed to designers and builders and may be written in terms of performance level specifications, system level specifications, detail level specifications, or a combination of these. Typically included (if not called out directly in the body of the contract) will be the Naval Administration requirement to the designer or builder to obtain Classification and various certifications required from other Certification Authorities, along with additional technical requirements the Naval Administration may have for specific systems, equipment, or materials. Those developing and approving ship specifications are responsible for avoiding conflict with the requirem

In [8]:
result = retrieval_grader.invoke({"question": question, "document": doc_txt})
result.binary_score

'yes'

In [9]:
grade_no_docs = [grade_no_doc for grade_no_doc in docs if retrieval_grader.invoke({"question": question, "document": doc_txt}).binary_score == 'no']
grade_no_docs

[]

In [10]:
grade_yes_docs = [grade_yes_doc for grade_yes_doc in docs if retrieval_grader.invoke({"question": question, "document": doc_txt}).binary_score == 'yes']
grade_yes_docs

[Document(metadata={'File Name': 'DNV Rules for Classification of Ships _2016_08_Introduction to Ship Classification', 'File Path': '/content/drive/MyDrive/Rules/DNV/DNV Rules for Classification of Ships _2016_08_Introduction to Ship Classification.pdf', 'First Division': 'Rules', 'Page': 4, 'Second Division': 'DNV'}, page_content="This page explains DNV Rules for Classification of Ships _2016_08_Introduction to Ship Classification, that belongs to catogories of Rules and DNV./ndisclosed to any party, apart fro m the national authorities in volved, without the owners consent. The Society also undertakes all reporting to national authorities required in connection with the safety certificates. B 200 Who needs classification? 201  Classification serves as verification system for a number of parties who have special interest in the safety and quality of ships, such as: — National authorities, who accept ships for registry, or let ships into thei r territorial waters, need assurance that t